In [1]:
%load_ext autoreload
%autoreload 2
!export CUDA_VISIBLE_DEVICES=5

import sys
    
import re
import os
import json
import torch
import numpy as np

sys.path.append('../')

In [93]:
from src.auto_chat.text_splitters import RecursiveCharacterTextSplitterAndFilterer, HTMLHeaderTextSplitterAndFilterer
from langchain_core.documents import Document

In [95]:
rts = RecursiveCharacterTextSplitterAndFilterer(
    min_words_drop=8,
    do_clean=False,
    **{
        "chunk_size": 1000,
        "chunk_overlap": 200,
        "separators": ["\n\n", "\n"]
    }
)

hts = HTMLHeaderTextSplitterAndFilterer(
    min_words_drop=8,
    do_clean=False,
    **{
        "headers_to_split_on": [
          [
            "h1",
            "Header 1"
          ],
          [
            "h2",
            "Header 2"
          ],
          [
            "h3",
            "Header 3"
          ]
        ]
      }
)

# list(reversed(["\n\n", "\n", " ", ""]))

In [96]:
ss = """asdfm asdf hi how are you? ?? ?? 



fsadf
oh how?



a
akbar akbarrrrrrr rr rr rr rr rr rr rr sef asdf rqwe wer sd
"""

# ss = "asdfm asdf hi how are you? ?? ?? "

html_string = """
<!DOCTYPE html>
<html>
<body>
    <div>
        <h1>Foo</h1>
        <p>Some intro text about Foo.</p>
        <div>
            <h2>Bar main section</h2>
            <p>Some intro text about Bar.</p>
            <h3>Bar subsection 1</h3>
            <p>Some text about the first subtopic of Bar.</p>
            <h3>Bar subsection 2</h3>
            <p>Some text about the second subtopic of Bar.</p>
        </div>
        <div>
            <h2>Baz</h2>
            <p>Some text about Baz</p>
        </div>
        <br>
        <p>Some concluding text about Foo</p>
    </div>
</body>
</html>
"""
docs = [
    Document(page_content=html_string)
]

In [92]:
print(rts.split_documents(documents=docs)[0].page_content)

asdfm asdf hi how are you? ?? ?? 

a
akbar akbarrrrrrr rr rr rr rr rr rr rr sef asdf rqwe wer sd


In [98]:
for i in hts.split_documents(documents=docs):
    print(i)

Splitting pages with HTML splitter: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 318.38it/s]

page_content='Some intro text about Foo.  \nBar main section Bar subsection 1 Bar subsection 2' metadata={'Header 1': 'Foo'}
page_content='Some text about the first subtopic of Bar.' metadata={'Header 1': 'Foo', 'Header 2': 'Bar main section', 'Header 3': 'Bar subsection 1'}
page_content='Some text about the second subtopic of Bar.' metadata={'Header 1': 'Foo', 'Header 2': 'Bar main section', 'Header 3': 'Bar subsection 2'}


In [ ]:
from datasets import Dataset
from ragas.metrics import context_precision
from ragas import evaluate

data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    'contexts' : [['The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles,'],
    ['The Green Bay Packers...Green Bay, Wisconsin.','The Packers compete...Football Conference']],
    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']
}
dataset = Dataset.from_dict(data_samples)
score = evaluate(dataset,metrics=[context_precision])
score.to_pandas()
